In [32]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint

In [23]:
df = pd.read_csv('titanic.csv')
df.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


In [24]:
df.shape

(887, 8)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887 entries, 0 to 886
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Survived                 887 non-null    int64  
 1   Pclass                   887 non-null    int64  
 2   Name                     887 non-null    object 
 3   Sex                      887 non-null    object 
 4   Age                      887 non-null    float64
 5   Siblings/Spouses Aboard  887 non-null    int64  
 6   Parents/Children Aboard  887 non-null    int64  
 7   Fare                     887 non-null    float64
dtypes: float64(2), int64(4), object(2)
memory usage: 55.6+ KB


In [26]:
df.isnull().sum().sort_values()

Survived                   0
Pclass                     0
Name                       0
Sex                        0
Age                        0
Siblings/Spouses Aboard    0
Parents/Children Aboard    0
Fare                       0
dtype: int64

In [27]:
# Sélectionne les features (X) et la cible (y) :
X = df[['Pclass', 'Age', 'Siblings/Spouses Aboard', 'Parents/Children Aboard', 'Fare']]
y = df['Survived']

# Normalise les données :
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Divise les données en ensbles d'entraînemt et de test :
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Entraîne le mdèle DecisionTreeClassifier :
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(X_train, y_train)

# Prédict° sur l'ensble de test :
y_pred = dt_classifier.predict(X_test)

# Calcule et affiche l'accuracy score :
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Précis° du mdèle sur l'ensble de test : {test_accuracy:.4f}")

Précis° du mdèle sur l'ensble de test : 0.6348


In [28]:
# GridSearch pr optimisat° des hyperpramètres :
# Défini la grille des hyperpramètres :
param_grid = {
# de 1 à 50 pr max_depth :
    'max_depth': list(range(1, 51)),
# de 1 à 15 pour min_samples_leaf :
    'min_samples_leaf': list(range(1, 16)),
# vleurs spé pr min_samples_split :
    'min_samples_split': [2, 5, 7, 10, 15, 30]  
}

# Initialise GridSearchCV avec le classifieur DecisionTreeClassifier :
grid_search = GridSearchCV(estimator=dt_classifier, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Effectue la rcherche sur la grille :
grid_search.fit(X_train, y_train)

# Affiche les meilleurs pramètres trvés :
print(f"Meilleurs paramètres trouvés par GridSearch : {grid_search.best_params_}")

# Évalue le mdèle optimisé sur l'ensble de test :
best_model = grid_search.best_estimator_
y_pred_optimized = best_model.predict(X_test)

# Calcule et affiche l'accuracy score du mdèle optimisé sur le jeu de test :
optimized_test_accuracy = accuracy_score(y_test, y_pred_optimized)
print(f"Précis° du mdèle optimisé sur l'ensble de test : {optimized_test_accuracy:.4f}")

Meilleurs paramètres trouvés par GridSearch : {'max_depth': 4, 'min_samples_leaf': 1, 'min_samples_split': 2}
Précis° du mdèle optimisé sur l'ensble de test : 0.6966


C:\Users\sebla\anaconda3\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [29]:
# Les meilleurs valeurs que définit GridSearch st : 4, 1 et 2.

In [37]:
# Définis l'espace des hyperpramètres pr RandomizedSearch :
param_dist = {
# de 1 à 50 pr max_depth
    'max_depth': randint(1, 51),  
# de 1 à 15 pr min_samples_leaf
    'min_samples_leaf': randint(1, 16),  
# vleurs spéc pr min_samples_split
    'min_samples_split': [2, 5, 7, 10, 15, 30]  
}
# Initialise RandomizedSearchCV avec 200 itérat° et 5 plis pr la validat° croisée :
random_search = RandomizedSearchCV(estimator=dt_classifier, param_distributions=param_dist, 
                                   n_iter=200, cv=5, scoring='accuracy', random_state=42, n_jobs=-1)

# Effectue la rcherche aléatoire :
random_search.fit(X_train, y_train)

# Affiche les meilleurs pramètres trvés par RandomizedSearchCV :
print(f"Meilleurs pramètres trvés par RandomizedSearch : {random_search.best_params_}")

# Évalue le mdèle optimisé sur l'ensble de test :
best_model_random = random_search.best_estimator_
y_pred_random = best_model_random.predict(X_test)

# Calcule et affiche l'accuracy score du mdèle optimisé sur le jeu de test :
optimized_test_accuracy_random = accuracy_score(y_test, y_pred_random)
print(f"Précis° du mdèle optimisé sur l'ensble de test (RandomSearch) : {optimized_test_accuracy_random:.4f}")

Meilleurs pramètres trvés par RandomizedSearch : {'max_depth': 5, 'min_samples_leaf': 5, 'min_samples_split': 7}
Précis° du mdèle optimisé sur l'ensble de test (RandomSearch) : 0.7022


C:\Users\sebla\anaconda3\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [ ]:
# Avec RandomSearch, les meilleures vleurs des hyperpramètres sont : 5, 5, 7.